In [1]:
import numpy as np
import cv2
import csv
import math
import numpy as np
import cv2
import csv
import math
import pandas as pd
import os

# pan 0 truck 1 ped 2 tilt 3 zoom 4
#w 새로 a는 이어서
        
r = np.zeros((300,600,1), np.uint8) + 255
g = np.zeros((300,600,1), np.uint8) + 255
b = np.zeros((300,600,1), np.uint8) + 255
rgb = np.zeros((300,600,3), np.uint8) + 255
    
def draw_flow_b(img,flow, step= 25,black=False):
        
        global width, height, cnt, r,g,b,rgb, rgb_pre,r_pre,g_pre,b_pre,x_to,y_to,to_pre

        
        
        h, w = img.shape[:2]
        
        y,x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
        
        fx, fy = flow[y,x].T
        
        
        lines = np.vstack([x , y , x+fx , y+fy ]).T.reshape(-1,2,2)
        #lines = np.int32(lines+0.5)
        lines = np.int32(lines)
        
        vis = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

     
        cv2.polylines(b,lines,0,(255,0,0),10)
        
        
        return vis
    
def draw_flow_g(img,flow, step= 25,black=False):
        
        global width, height, cnt, r,g,b,rgb, rgb_pre,r_pre,g_pre,b_pre,x_to,y_to,to_pre

        
        
        h, w = img.shape[:2]
        
        y,x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
        
        fx, fy = flow[y,x].T
        
        
        lines = np.vstack([x , y , x+fx , y+fy ]).T.reshape(-1,2,2)
        #lines = np.int32(lines+0.5)
        lines = np.int32(lines)
        
        vis = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

     
        if black:
            vis = np.zeros((height,width,3),np.uint8)
        cv2.polylines(vis,lines,0,(0,255,0),10)
        
        cv2.polylines(g,lines,0,(0,255,0),10)
        
        return vis
    
def draw_flow_r(img,flow, step= 25,black=False):
        
        global width, height, cnt, r,g,b,rgb, rgb_pre,r_pre,g_pre,b_pre,x_to,y_to,to_pre

        
        
        h, w = img.shape[:2]
        
        y,x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
        
        fx, fy = flow[y,x].T
        
        
        lines = np.vstack([x , y , x+fx , y+fy ]).T.reshape(-1,2,2)
        #lines = np.int32(lines+0.5)
        lines = np.int32(lines)
        
        vis = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

     
        if black:
            vis = np.zeros((height,width,3),np.uint8)
        cv2.polylines(vis,lines,0,(0,255,0),10)
        cv2.polylines(r,lines,0,(0,0,255),10)
        
        return vis

In [ ]:
import os
import math 
import numpy as np
import cv2
listing = os.listdir("train")
count = 0
for folder in listing:
    #os.makedirs("data_frames/"+ folder+"/")
    for videoname in os.listdir("train/"+ folder):
        global width, height
        print("train/" + folder + "/" + videoname)
        cap = cv2.VideoCapture("train/" + folder + "/" + videoname)
        #fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
        ret, prev = cap.read()
        #fps = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        success=1
        prev = cv2.resize(prev, dsize=(1200,800), fx=0, fy=0)
        prevgray = cv2.cvtColor(prev,cv2.COLOR_BGR2GRAY)
        frame_save = []
        x1 = 0
        y1 = 0 
        leng1 = 0

       
        print("fps",fps)
        show_hsv = False
        show_glitch = False
        cur_glitch = prev.copy()

        blackscreen= False

        prevshot1 = prevgray[100 : 700, 0 : 1200]
        
        image_count = 1

        cnt = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_save.append(frame)
        #cv2.imshow("flow1",frame)
        ch = cv2.waitKey(60) & 0xFF
        if ch==27:
            break
        k = np.array(frame_save).shape[0]   
        z = 11*3+1
        print("k",k,"k-z",k-z)
        for i in range(0,k-z):
            count = count+1
            print(count)
            frame1 = frame_save[i]
            frame2 = frame_save[i+11]
            frame3 = frame_save[i+22]
            frame4 = frame_save[i+33]

            frame1 = cv2.resize(frame1, dsize=(600,300), fx=0, fy=0)
            frame2 = cv2.resize(frame2, dsize=(600,300), fx=0, fy=0)
            frame3 = cv2.resize(frame3, dsize=(600,300), fx=0, fy=0)
            frame4 = cv2.resize(frame4, dsize=(600,300), fx=0, fy=0)

            gray1 = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
            gray3 = cv2.cvtColor(frame3,cv2.COLOR_BGR2GRAY)
            gray4 = cv2.cvtColor(frame4,cv2.COLOR_BGR2GRAY)

            flow1 = cv2.calcOpticalFlowFarneback(gray1,gray2, 0.0 , 0.5 , 3 , 15 , 3 , 5 , 1.1 , 0 )
            flow2 = cv2.calcOpticalFlowFarneback(gray2,gray3, 0.0 , 0.5 , 3 , 15 , 3 , 5 , 1.1 , 0 )
            flow3 = cv2.calcOpticalFlowFarneback(gray3,gray4, 0.0 , 0.5 , 3 , 15 , 3 , 5 , 1.1 , 0 )



            frame_fist = draw_flow_b(gray2,flow1,black=blackscreen)
            frame_second= draw_flow_g(gray3,flow2,black=blackscreen)
            frame_third = draw_flow_r(gray4,flow3,black=blackscreen)


            rgb = cv2.merge((b, g, r))

            if folder == 'pan':
                path = 'dataset/11_frames/train/pan'
            elif folder =='truck':
                path = 'dataset/11_frames/train/truck'
            elif folder =='tilt':
                path = 'dataset/11_frames/train/tilt'
            elif folder =='pedestal':
                path = 'dataset/11_frames/train/pedestal'
            elif folder =='zoom_in':
                path = 'dataset/11_frames/train/zoom_in'
            elif folder =='zoom_out':
                path = 'dataset/11_frames/train/zoom_out'
            
            cv2.imwrite(os.path.join(path , f"{count}.png"), rgb)


            r = np.zeros((300,600,1), np.uint8) + 255
            g = np.zeros((300,600,1), np.uint8) + 255
            b = np.zeros((300,600,1), np.uint8) + 255
            rgb = np.zeros((300,600,3), np.uint8) + 255    
        
        frame_save.clear()
        print("end") 